<a href="https://colab.research.google.com/github/Piontk/Autonomous-Database-RAG/blob/main/Sobre_RAG_e_Vector_Stores_na_Pr%C3%A1tica_com_Autonomous_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchain-openai langchain-oracledb pypdf oracledb python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_oracledb.vectorstores.oraclevs import OracleVS
from dotenv import load_dotenv
import oracledb

load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
CONNECT_STRING = os.getenv("CONNECT_STRING")
WALLET_PASSWORD = os.getenv("WALLET_PASSWORD")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def get_vector_database() -> OracleVS:
    """Cria e retorna o objeto Oracle Vector Store."""
    try:
        connection = oracledb.connect(user=DB_USER,
                              password=DB_PASSWORD,
                              dsn=CONNECT_STRING,
                              config_dir="/content/wallet",
                              wallet_location="/content/wallet",
                              wallet_password=WALLET_PASSWORD )
        print("Connection successful!")
    except Exception as e:
        print("Connection failed!")

    TABLE_NAME = "ORACLE_DOCS"

    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

    oraclevs = OracleVS(
        client=connection,
        table_name=TABLE_NAME,
        embedding_function=embeddings,
    )

    return oraclevs

def ingest_pdf(pdf_folder_path: str) -> list:
    """
    Lê todos os PDFs em um diretório e os divide em chunks.
    """
    all_docs = []
    print(f"Lendo PDFs do diretório: {pdf_folder_path}")
    for filename in os.listdir(pdf_folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(pdf_folder_path, filename)
            try:
                loader = PyPDFLoader(file_path)
                documents = loader.load()
                all_docs.extend(documents)
                print(f"  - Carregado: {filename} ({len(documents)} páginas)")
            except Exception as e:
                print(f"Erro ao carregar o arquivo {filename}: {e}")

    if not all_docs:
        print("Nenhum documento PDF encontrado para processar.")
        return

    # Dividir os documentos em chunks. Esta é a parte crucial para manter o contexto.
    # O splitter tenta manter parágrafos e sentenças juntos.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,  # Tamanho do chunk
        chunk_overlap=600 # Sobreposição para não perder contexto entre chunks
    )
    chunks = text_splitter.split_documents(all_docs)
    print(f"\nTotal de documentos dividido em {len(chunks)} chunks para armazenamento.")

    return chunks

def ingest_pdf_chunks(chunks: list):
    """
    Insere os chunks de texto no vector store.
    """
    if not chunks:
        print("Nenhum chunk para processar.")
        return

    print(f"Inserindo {len(chunks)} chunks na Vector Store...")

    # Obter a vector store e adiciona os chunks
    vector_store = get_vector_database()

    print("Iniciando a inserção dos chunks na Vector Store (pode levar alguns minutos)...")
    vector_store.add_documents(chunks)

    print("\nIngestão de PDFs concluída com sucesso!")

def create_analysis_chain():
    """
    Cria a cadeia de processamento (RAG chain) para analisar o AWR.
    """
    vector_store = get_vector_database()
    retriever = vector_store.as_retriever(search_kwargs={"k": 5}) # Busca os 5 chunks mais relevantes

    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2, api_key=OPENAI_API_KEY)

    # Este é o prompt que guiará o modelo. É a parte mais importante para a qualidade da resposta.
    template = """
    Você é um assistente especialista em Oracle Database.
    Seu papel é responder perguntas técnicas sobre Oracle de forma clara, precisa e fundamentada.
    Você tem acesso a uma base de conhecimento contendo documentações oficiais da Oracle.

    INSTRUÇÕES:
    1. Sempre baseie suas respostas nas documentações Oracle recuperadas do contexto fornecido.
    2. Responda em português técnico e direto, mas explique termos quando necessário.
    3. Se houver código SQL, PL/SQL ou comandos administrativos, use a sintaxe do Oracle 23ai (ou a versão mencionada no contexto).
    4. Quando a resposta envolver opções, explique prós e contras brevemente.
    5. Se o contexto recuperado não contiver informação suficiente, diga claramente:
       "Não encontrei essa informação na documentação disponível."
    6. Não invente comandos ou recursos que não existem.

    FORMATO DA RESPOSTA:
    - **Explicação resumida** da resposta.
    - **Detalhamento técnico** com base na documentação recuperada.
    - **Exemplo prático** (quando aplicável).

    CONTEXTOS DISPONÍVEIS:
    {context}

    PERGUNTA DO USUÁRIO:
    {question}

    RESPOSTA:
    """

    prompt = ChatPromptTemplate.from_template(template)

    # Cria a RAG chain usando o LangChain Expression Language (LCEL)
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [2]:
chunks = ingest_pdf('/content/pdfs')

/content
